# Set up the envirionment

In [ ]:
# The default envirionment comes loaded with many of the most common R packages
# Additional packages can  be installed into the R enviroinment using install.packages()
# Here, we install a few packages needed for the multivariable analysis
install.packages(c("car", "rms", "arm"))

In [ ]:
# We next load the required libraries
require(rms)
library(formula.tools)
library(gtools)
library(car)
library(arm)
library(tidyverse)


# Load Analysis Functions

In [ ]:
# We define a set of analysis functions that will implement a multivariable analysis
# These may be copy/pasted into your own code unmodified, if desired

# This formats p-values to three decimal places, or for p-values <0.001 displays "<0.001"
pfmt <- function(pvals) {
  res <- c()
  for(i in 1:length(pvals)) {
    if( pvals[i] < 0.001 ) {
      res <- c(res, "<0.001")
    } else {
      formatted <- format(round(pvals[i], 3), nsmall = 3)
      res <- c(res, formatted)
    }
  }
  
  return(res)
}  

# Accepts a logistic regression fit, calculates the odds ratio, extracts
# the 95% confidence interval and extracts/formats p-values
extractORandCI <- function(fit) {
    # Extract log or and log ci from the fit
    or <- coef(fit)
    ci <- suppressMessages(confint(fit))
    
    rn <- attr(or, 'names')
    
    # Exponentiate to get actual or and CI
    or <- exp(or)
    ci <- exp(ci)
    
    # Extract the p value
    p <- coef(summary(fit))[,4]
    
    #Format the p-value
    st <- stars.pval(p)
    #p <- format.pval(p, digits=2, nsmall=3, eps=0.001)
    p <- pfmt(p)
    
    # Format the OR and CI
    or <- format(round(or, 2), nsmall = 2)
    ci <- format(round(ci, 2), nsmall = 2)
    
    # Construct a pretty version of the or and ci
    prty <- paste0(or, ' (', ci[,1], ' - ', ci[,2], ')')
    
    res <- data.frame(varia=rn, or=prty, p=p, stars=st)
    
    return(res)
  }


# This is a complicated function that accepts a model function and a dataset
# and calculates two things:
# First, it calculates bivariable logistic regressions between the outcome variable
# in the model, and each of the predictor variables in isolation
# Then, it calculates a full multivariable logistric regression using the model
# 
calculateModels <- function(f, dataset) {
  # Extract the RHS (predictor) variables
  unimodels <- rhs.vars(f)
  
  # Get the LHS (response) variable
  response <- lhs.vars(f)
  
  # Perform bivariable analysis
  unires <- data.frame()
  # For each variable
  for(i in 1:length(unimodels)) {
    univar <- unimodels[i] # Get the variable
    fituni <- glm(as.formula(paste0(response, ' ~ ',univar)), data=dataset, family="binomial") # Dynamically construct the formula and run logistic regression
    crudeor <- extractORandCI(fituni) # Get the odds ratios and CI's
    crudeor <- tail(crudeor, -1) # Disregard the intercept term in each crude analysis
    unires <- rbind(unires, crudeor) # Add to the accumulating table
  }
  unires$fit.type="bivariable"
  
  # Perform multivariable analysis
  multifit <- glm(f, data=dataset, family="binomial") # Perform the fit
  multires <- extractORandCI(multifit) # Get the OR and CI's from the fit
  multires <- tail(multires, -1) # Disregard the intercept term in each crude analysis
  multires$fit.type="multivariable" 

  # Append the multivariable and bivariable data tables  
  full_result <- rbind(unires, multires)
  
  # Reformat this table so that rows corrospond to a variable and the bivariable and multivariables
  # odds ratios show up on the left/right of the table, respectively
  full_result <- full_result %>%
    pivot_wider(names_from=fit.type, values_from=c('or', 'p', 'stars')) %>%
    dplyr::select(-c(stars_bivariable)) %>%
    dplyr::relocate(or_multivariable, .after = p_bivariable)

  # Calcualte a chi-square for the model
  mchi <- with(multifit, pchisq(null.deviance - deviance, df.null - df.residual, lower.tail = FALSE))
  
  # Calculate variance inflation factors (VIF) which check for multicollinearity
  vif <- car::vif(multifit)
  
  # Get some performance statistics (e.g., C-statistic, etc) for the multivariable fit
  multistats <- lrm(formula=f, data=dataset)$stats
  
  # Return all of this to the user"
    # datatable: This is the summary table containing bivariable and multivariable results
    # datatable_multi: This is just the multivariable data table in isolation
    # multi_chisq: The chi-square for the multivariable fit
    # vif: The variance inflation factors for each variable in the multivariable fit to check for multicollinearity
    # multifit: The actual logistic regression fit object (in case downstream analysis is needed)
    # multistats: The performance statistics of the multivariable fit
  return(list(datatable=full_result, datatable_multi=multires, multi_chisq=mchi, vif=vif, multifit=multifit, multistats=multistats))
}

# Load Data

In [ ]:
# This snippet is almost entirely drawn from the All of Us snippets library
# (see the toolbar at the top of the Researcher Workbench's jupyter envirionment
# for this and other snippets)

# In this analysis, we loaded a CSV file created by python called processed-data.csv
name_of_file_in_bucket <- 'processed-data.csv'

# This gets the name of the Google bucket that we are storing data in
my_bucket <- Sys.getenv('WORKSPACE_BUCKET')

# This copies the file we want from our workspace bucket to the local envirionment
system(paste0("gsutil cp ", my_bucket, "/data/", name_of_file_in_bucket, " ."), intern=T)

# This actually reads the data in
dat_raw  <- read_csv(name_of_file_in_bucket)


# The Analysis

In [ ]:
# Get the colums we need
dat <- dat_raw %>% select(all_of(c('vbt', 'fi_ranout', 'fi_worried', 'social_risk', 'metformin', 'age', 'male')))

# Set the data types for ordinal variables
dat$fi_ranout <- ordered(dat$fi_ranout, levels=c('Never true', 'Sometimes true', 'Often true')) # Experience of food insecurity
dat$fi_worried <- ordered(dat$fi_worried, levels=c('Never true', 'Sometimes true', 'Often true')) # Worry about food insecurity

# Construct some binary variables as predictors in the regression
dat$fi_ranout.bin <- (dat$fi_ranout > 'Never true')
dat$fi_worried.bin <- (dat$fi_worried > 'Never true')

# Construct Vitamin B12 levels 
dat$vbt200 = (dat$vbt < 200)
dat$vbt300 = (dat$vbt < 300)
dat$vbt400 = (dat$vbt < 400)
dat$vbt500 = (dat$vbt < 500)

# Here is a very simple example of how to run a logistic regression on this data
model <- glm((vbt < 300) ~ (fi_ranout > 'Never true') + age + metformin + male, dat=dat, family='binomial')

# Display the model data (does not give odds ratios or CI's directly, but you can view the p-values for your
# significant variables)
summary(model)

In [ ]:
# What we'll actually do is instead use the calculateModels function, from above, to fit 
# multivariable and bivariable models all in one step

# Calculate the primary outcome (VitB12 < 300 pg/mL)
model300 <- calculateModels(vbt300 ~ fi_ranout.bin + age + metformin + male, dat)

In [ ]:
# Sensitivity: Explore alternative Vitamin B12 thresholds
model200 <- calculateModels(vbt200 ~ fi_ranout.bin + age + metformin + male, dat)
model400 <- calculateModels(vbt400 ~ fi_ranout.bin + age + metformin + male, dat)
model500 <- calculateModels(vbt500 ~ fi_ranout.bin + age + metformin + male, dat)

In [ ]:
# Sensitivity: Using worried instead of experience
model200w <- calculateModels(vbt200 ~ fi_worried.bin + age + metformin + male, dat)
model300w <- calculateModels(vbt300 ~ fi_worried.bin + age + metformin + male, dat)
model400w <- calculateModels(vbt400 ~ fi_worried.bin + age + metformin + male, dat)
model500w <- calculateModels(vbt500 ~ fi_worried.bin + age + metformin + male, dat)

In [ ]:
# Sensitivity: Using composite of experience and/or worried
model200c <- calculateModels(vbt200 ~ social_risk + age + metformin + male, dat)
model300c <- calculateModels(vbt300 ~ social_risk + age + metformin + male, dat)
model400c <- calculateModels(vbt400 ~ social_risk + age + metformin + male, dat)
model500c <- calculateModels(vbt500 ~ social_risk + age + metformin + male, dat)

# Results for the Primary Outcome

In [ ]:
# Display the summary data table for the multivariable analysis
model300$datatable

# Results for the sensitivity analyses

## Different thresholds for Vitamin B12 deficiency

In [ ]:
model200$datatable
model400$datatable
model500$datatable

## Worry about food insecurity instead of experience of food insecurity

In [ ]:
model200w$datatable
model300w$datatable
model400w$datatable
model500w$datatable

## Sensitivity: Composite variable of worry and/or experience of food insecurity

In [ ]:
model200c$datatable
model300c$datatable
model400c$datatable
model500c$datatable

## An additional sensitivity analysis changing age to a 4-level variable instead of a continuous one

In [ ]:
dat$age.quant = quantcut(dat$age, q = 4)
model200aq <- calculateModels(vbt200 ~ fi_ranout.bin + age.quant + metformin + male, dat)
model300aq <- calculateModels(vbt300 ~ fi_ranout.bin + age.quant + metformin + male, dat)
model400aq <- calculateModels(vbt400 ~ fi_ranout.bin + age.quant + metformin + male, dat)
model500aq <- calculateModels(vbt500 ~ fi_ranout.bin + age.quant + metformin + male, dat)

In [ ]:
model200aq$datatable
model300aq$datatable
model400aq$datatable
model500aq$datatable

# Data about the sample

In [ ]:
# This shows some examples of how to extract information about the sample

# Get the total number of rows in the sample
nrow(dat)

# Show the total number of people with various levels of Vitamin B12 deficiency

# This basically does "For each person, if vitamin B12 is less than 200, output a 1, otherwise output a zero. 
# Then sum this up." It counts the total numbe rof people with Vitamin B12 < 200.
sum(ifelse(dat$vbt < 200, 1, 0))

# Same thing for 300 pg/mL, 400 pg/mL and 500 pg/mL
sum(ifelse(dat$vbt < 300, 1, 0))
sum(ifelse(dat$vbt < 400, 1, 0))
sum(ifelse(dat$vbt < 500, 1, 0))

In [ ]:
# Show the distribution of male and female participant
table(dat$male, exclude=NULL)

In [ ]:
# Alternatively, you can do the same thing with dplyr, which looks nicer (if somewhat more verbose)
dat %>%
    group_by(male) %>%
    dplyr::summarize(count=n())

In [ ]:
# Show the age distribution
table(dat$age.quant, exclude=NULL)

In [ ]:
# Metformin distribution
table(dat$metformin, exclude=NULL)

In [ ]:
# Get min, Q1, median, Q3, max of ages
quantile(dat$age, exclude=NULL)

In [ ]:
# Show the distribution for experience of food insecurity (and the percentages)
table(dat$fi_ranout)
table(dat$fi_ranout) / sum(table(dat$fi_ranout)) * 100

In [ ]:
# Show the distribution for worry about food insecurity (and the percentages)
table(dat$fi_worried)
table(dat$fi_worried) / sum(table(dat$fi_worried)) * 100

In [ ]:
# Show the racial distribution
table(dat_raw$race, exclude=NULL)

In [ ]:
# Show the ethnicity distribution
table(dat_raw$ethnicity, exclude=NULL)

In [ ]:
# Show the sex at birth distribution
table(dat_raw$sex_at_birth, exclude=NULL)

In [ ]:
# Show the joint distribution of sex at birth and gender
# This is hard to read if you use 'table' so instead we'll use the dplyr approach
dat_raw %>%
    group_by(gender, sex_at_birth) %>%
    dplyr::summarize(count=n())

In [ ]:
# You could have calcualted the joint distribution with table, but it's hard to read:
with(dat_raw, table(gender, sex_at_birth, exclude=NULL))

In [ ]:
# Figure out which version of R we're using
version

In [ ]:
# Check on the recorded units of the Vitamin B12 serum levels
dat_raw %>%
    group_by(unit_concept_name) %>%
    dplyr::summarize(count=n())